In [7]:
import pandas as pd
import re
import numpy as np
import nltk
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('vader_lexicon')
# from wordcloud import WordCloud
import matplotlib.pyplot as plt
stop_words = set(stopwords.words('english')) 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import textblob
import matplotlib.pyplot as plt
import bs4
import itertools
from sklearn.model_selection import train_test_split
import fasttext
from fasttext import load_model
from sklearn.utils import shuffle
# df = shuffle(df)

In [8]:
tweet_df = pd.read_csv('training.1600000.processed.noemoticon.csv', names=["target", "ids", "date", "flag", "user", "text"])
tweet_df = shuffle(tweet_df)
tweet_df.head(30)

,target,ids,date,flag,user,text
1565017,4,2187438806,Mon Jun 15 20:16:57 PDT 2009,NO_QUERY,kb3mkd,@TPO_Hisself
703700,0,2255787480,Sat Jun 20 11:47:21 PDT 2009,NO_QUERY,iShatara,Damn I been up since 8 and ain even ate!!!
227324,0,1978035085,Sat May 30 22:16:44 PDT 2009,NO_QUERY,Ruthycpr,@richsolarstone omg i miss the glee club so mu...
618501,0,2227541847,Thu Jun 18 13:19:39 PDT 2009,NO_QUERY,LadybotRocks,A bit ticked off at digital Tv revolution. Can...
1142299,4,1977285078,Sat May 30 20:35:22 PDT 2009,NO_QUERY,smileybubbbles,Debating with a high school debate teacher wha...
831130,4,1557401611,Sun Apr 19 03:53:40 PDT 2009,NO_QUERY,rom,Now we're talking! This can be called 'raining...
795256,0,2327294388,Thu Jun 25 08:09:57 PDT 2009,NO_QUERY,MarianAllen,@russmarshalek Sorry to hear you're not havin...
295861,0,1996972900,Mon Jun 01 16:01:27 PDT 2009,NO_QUERY,tails479,Thank God I Am Pretty. The occasional free dr...
880052,4,1685571213,Sun May 03 01:00:15 PDT 2009,NO_QUERY,lumpadoodles,"back to work i go, hoopefully i will stop drop..."
1400393,4,2054381036,Sat Jun 06 07:13:35 PDT 2009,NO_QUERY,J_Monopoly,@iheartdilla Lol..I wish every1 would stop say...


In [9]:
tweet_df['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

In [10]:
tweet_df['target'].replace({0:'negative', 4:'positive'}, inplace=True)
tweet_df.head()

,target,ids,date,flag,user,text
1565017,positive,2187438806,Mon Jun 15 20:16:57 PDT 2009,NO_QUERY,kb3mkd,@TPO_Hisself
703700,negative,2255787480,Sat Jun 20 11:47:21 PDT 2009,NO_QUERY,iShatara,Damn I been up since 8 and ain even ate!!!
227324,negative,1978035085,Sat May 30 22:16:44 PDT 2009,NO_QUERY,Ruthycpr,@richsolarstone omg i miss the glee club so mu...
618501,negative,2227541847,Thu Jun 18 13:19:39 PDT 2009,NO_QUERY,LadybotRocks,A bit ticked off at digital Tv revolution. Can...
1142299,positive,1977285078,Sat May 30 20:35:22 PDT 2009,NO_QUERY,smileybubbbles,Debating with a high school debate teacher wha...


In [11]:
def preprocess_tweet(tweet):
    #Preprocess the text in inplace=e tweet
    #arguments: tweet = a single tweet in form of string 
    #convert the tweet to lower case
    tweet = tweet.lower()
    #convert all urls to sting "URL"
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','', tweet)
    #convert all @username to "AT_USER"
    tweet = re.sub('@[^\s]+','', tweet)
    #convert "#topic" to just "topic"
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #convert RT to whote space
    tweet = re.sub(r'rt', '', tweet)
    # remove punctuation
    tweet = re.sub('[\W]+', ' ', tweet)
    #correct all multiple white spaces to a single white space
    tweet = re.sub('[\s]+', ' ', tweet)
    return tweet.strip()

tweet_df['cleaned_text'] = tweet_df['text'].apply(preprocess_tweet)

In [12]:
def transform_label(target):
    target = '__label__' +str(target)
    return target

tweet_df['label'] = tweet_df['target'].apply(transform_label)

In [13]:
tweet_df['label_text'] = tweet_df['label']+' ' +tweet_df['cleaned_text']
tweet_df.head()

,target,ids,date,flag,user,text,cleaned_text,label,label_text
1565017,positive,2187438806,Mon Jun 15 20:16:57 PDT 2009,NO_QUERY,kb3mkd,@TPO_Hisself,,__label__positive,__label__positive
703700,negative,2255787480,Sat Jun 20 11:47:21 PDT 2009,NO_QUERY,iShatara,Damn I been up since 8 and ain even ate!!!,damn i been up since 8 and ain even ate,__label__negative,__label__negative damn i been up since 8 and a...
227324,negative,1978035085,Sat May 30 22:16:44 PDT 2009,NO_QUERY,Ruthycpr,@richsolarstone omg i miss the glee club so mu...,omg i miss the glee club so much me and stu us...,__label__negative,__label__negative omg i miss the glee club so ...
618501,negative,2227541847,Thu Jun 18 13:19:39 PDT 2009,NO_QUERY,LadybotRocks,A bit ticked off at digital Tv revolution. Can...,a bit ticked off at digital tv revolution can ...,__label__negative,__label__negative a bit ticked off at digital ...
1142299,positive,1977285078,Sat May 30 20:35:22 PDT 2009,NO_QUERY,smileybubbbles,Debating with a high school debate teacher wha...,debating with a high school debate teacher wha...,__label__positive,__label__positive debating with a high school ...


In [14]:
train, test = train_test_split(tweet_df['label_text'], test_size=0.2, random_state=1)

In [15]:
def transform_data_format(data, data_label='data'):
    
    data = data.tolist()
    with open('{}.txt'.format(data_label), 'a', encoding='utf-8') as file:
        for line in data:
            file.writelines('{}\n'.format(line))
            
    with open('{}.txt'.format(data_label), 'r', encoding='utf-8') as f:
        txt_list = f.readlines()
        
    return txt_list

train_list = transform_data_format(train, 'train_data')
text_list = transform_data_format(test, 'test_data')

In [16]:
text_list[1:10]

['__label__negative so i had a dream last night i remember a sign which clearly told me where to get a job i can t rememer what the sign said\n',
 '__label__negative ohh poor sickly you hugs hope you feel a little better soon\n',
 '__label__negative it is raining again\n',
 '__label__negative wish i was in la right now\n',
 '__label__negative nala olowalu still has a full tummy from bread basket 2\n',
 '__label__positive site doesn t seem to want to load up they must be getting a lot of hits\n',
 '__label__negative time for some sleep hav to actually do some work tmrw\n',
 '__label__negative in oxford that month\n',
 '__label__positive time for a cup of tea and fruit bagels i m going to turn the day around\n']

In [20]:
model = fasttext.train_supervised('train_data.txt', epoch=20)

# model.save_model('model_2_140.ftz')

In [21]:
labels = model.test('test_data.txt')
labels

(640000, 0.81174375, 0.81174375)